## Import Libraries

In [59]:
from nltk.tokenize import word_tokenize
import pyarabic.araby as araby
import re
import pandas as pd

## Text Preprocessing
------------
- Readthe document /query text.
- Create an Indices Vector.
- Normalize text (Part 1 ): remove shaddah and tanween
- Remove Stop Words in their different forms ( Prefix + Suffix, Stop Word, Prefix + Stop
Word, Stop Word + Suffix, Prefix + Stop Word + Suffix
- Normalize text (Part 2 ) : unification of letters ’ forms

### Step 1: Read the document 


In [60]:
text = """الإيمانُ ..وأثره على الصحة النفسية 
كان علم النفس دائماً فخوراً كعلم بالجامعات بتقاليده الدنيوية )غير
الدينية( الملتزمة بالتنوير. وكان -على الدوام- من ضمن هذه
التقاليد وجود شك واضح بكل أشكال التدين  كما يصف بيرنارد
وغروم"""

In [61]:
# Dataframe 
data = {}

### Step 2: Create an Indices Vector


In [62]:
indices_vector = {}
tokens = word_tokenize(text)
for i, token in enumerate(tokens):
    indices_vector[token] = i
    
print(indices_vector)

{'الإيمانُ': 0, '..': 1, 'وأثره': 2, 'على': 3, 'الصحة': 4, 'النفسية': 5, 'كان': 6, 'علم': 7, 'النفس': 8, 'دائماً': 9, 'فخوراً': 10, 'كعلم': 11, 'بالجامعات': 12, 'بتقاليده': 13, 'الدنيوية': 14, ')': 15, 'غير': 16, 'الدينية': 17, '(': 18, 'الملتزمة': 19, 'بالتنوير': 20, '.': 21, 'وكان': 22, '-على': 23, 'الدوام-': 24, 'من': 25, 'ضمن': 26, 'هذه': 27, 'التقاليد': 28, 'وجود': 29, 'شك': 30, 'واضح': 31, 'بكل': 32, 'أشكال': 33, 'التدين': 34, 'كما': 35, 'يصف': 36, 'بيرنارد': 37, 'وغروم': 38}


##### Step 3:  Normalize text (Part 1) - remove shaddah and tanween

In [63]:
def Normalize1(text): #remove shaddah and tanween
    #text = araby.strip_tashkeel(text)
    tokens = araby.tokenize(text, conditions=araby.is_arabicrange, morphs=araby.strip_tashkeel)
    return tokens

In [64]:
tokens  = Normalize1(text)

### Step 4: Remove Stop Words

In [65]:
def remove_stopwords(tokens):
    
    # Read stop words from file
    with open('stopwords.txt', 'r', encoding='utf-8') as f:
        stop_words = set([line.strip() for line in f.readlines()])

    # Remove stop words
    filtered_tokens = []
    
    #tokens = word_tokenize(text)
    #tokens = re.findall(r'\b\w+\b', text)
    
    for token in tokens:
        if token not in stop_words:
            filtered_tokens.append(token)
            
    return filtered_tokens

In [66]:
tokens = remove_stopwords(tokens)
data['Tokens'] = tokens.copy()

#### Step 5: Normalize text (Part 2 ) : unification of letters ’ forms

In [67]:
# Normalization Dictionary
Norm =  {   
            'ا' : ['أ','آ','ء'],
            'ي' : ['ى'],
            'ه': ['ة' ],
        }

In [68]:
def Normalize2(tokens):
    for i, token in enumerate(tokens):
        if token == 'الله':
            continue
        
        for key, values in Norm.items():
            for value in values:
                if value == 'ه' and tokens[i].endswith(value):
                    tokens[i] = tokens[i][:-1] + key
                elif value in tokens[i]:
                    tokens[i] = tokens[i].replace(value, key)
                    
    return tokens

In [69]:
tokens = Normalize2(tokens)
data['Normalized Tokens'] = tokens.copy()

### Removal Suffix

In [70]:
suffixes = ['ناكمو', 'ناهما', 'كموها','موهم', 'موهن', 'ناكم', 'اءكم', 'اءنا', 'اءهم', 'نوهن', 'اؤهم', 'ونهم', 'ائهم', 'ناهم', 'ونكم', 'ونهم', 'ائنا', 'توهن', 'اتها', 'اتهم', 'ياته',
'تنا', 'نها', 'تان', 'ونه', 'اءه', 'ناه', 'هما', 'وها', 'نهم', 'وهم', 'وني', 'وهن', 'تها', 'تهم', 'نكم', 'هات', 'تان', 'تهن', 'وكم', 'ونه', 'ونك', 'انك', 'ائك', 'يهم',
'ان', 'ين', 'ون', 'نا', 'تم', 'تا', 'ون', 'وا', 'ات', 'ان', 'ين', 'تن', 'كم', 'هن', 'نا', 'يا', 'ها', 'تم', 'كن', 'ني', 'وا', 'ما', 'هم', 'ها', 'وك', 'وت', 'وة', 'ية',
'ه', 'ي', 'ك', 'ت', 'ا', 'ن']

In [71]:
def apply_suffix_rules(word):
   for suffix in suffixes:
        if word.endswith(suffix) and (len(word) - len(suffix)) >= 3:
            if suffix == "ات":
                if not word.startswith("ال") or len(word) > 5:
                    word = word[:-2] + "ه"
                    break
                else:
                    word = word[:-2]
                    break
            elif suffix == "اه":
                word = word[:-2] + "ي"
                break
            elif word.startswith("ال") and suffix not in ["ه", "ي"] or (len(word) - len(suffix)) <= 5:
                return word

            else:
                word = word[:-len(suffix)]
                break
   return word


In [72]:
def remove_allsuffixes(tokens):
    new_words = []
    for token in tokens:
      new_words.append(apply_suffix_rules(token))        
    # Join the words back into a text.

    return new_words

In [73]:
tokens = remove_allsuffixes(tokens)
data['Suffex Removed'] = tokens.copy()

### Removal Prefix

In [74]:
Prefixes =  ['وليست', 'فليست', 'اليست', 'افلست',
'وبال', 'فبال', 'لبال','بال', 'فال', 'وال', 'كال', 'ولل','بال','اتست', 'ويست', 'فاست', 'واست', 'انهم', 'والم', 'باست', 'الاس', 'كمست', 'والا', 'فاست',
'مست', 'ولت', 'فلي', 'فلن', 'فلل', 'فان', 'يست', 'است', 'تست', 'فلي', 'وسي', 'وسن', 'فلا', 'وست', 'بمس',
'لل', 'ال', 'اا', 'لي','او', 'اي', 'ان', 'في', 'فب', 'فت', 'لي', 'فن', 'وب', 'فا', 'ول', 'وو', 'أف', 'في', 'لا', 'ات', 'وي', 'وت', 'سي', 'سن', 'فا', 'وي', 'مم', 'كت', 'مت', 'مس', 'وست', 'بمس',
'ك','ل', 'ب', 'ف', 'س', 'و', 'ي', 'ت', 'ن', 'ا','م']


In [75]:
def remove_prefix(word):
    if word == "الله":
        return word
    elif word.startswith("اا"):
        return word[1:]
    elif len(word) >= 5 and word.startswith("وال"):
        return word[3:]
    elif len(word) > 4 and word.startswith("لل"):
        return word[2:]
    elif len(word) >= 4 and word.startswith("ال"):
        return word[2:]
    elif len(word) >= 3:  
        for pre in Prefixes:
          if word.startswith(pre):
              x = int(len(pre))
              return word[x:]
    return word

def remove_allprefixes(tokens):
    pref = []
    for worde in tokens:
       pref.append(remove_prefix(worde))        
    # Join the words back into a text. 
    return pref

In [76]:
tokens = remove_allprefixes(tokens)
data['Prefex Removed'] = tokens.copy()

### Remove Last Suffix

In [77]:
def removeLastSuffix(tokens):
    new_words = []
    for word in tokens:
        if word.endswith("ا"):
            new_words.append(word[:-1])
        else:
            new_words.append(word)
    return new_words


In [78]:
tokens = removeLastSuffix(tokens)
data['Last Suffex Removed'] = tokens.copy()

## Final Result

In [79]:
def remove_stopwordss(tokens):
    
    # Read stop words from file
    with open('stopwords.txt', 'r', encoding='utf-8') as f:
        stop_words = set([line.strip() for line in f.readlines()])

    # Remove stop words
    filtered_tokens = []
    
    #tokens = word_tokenize(text)
    #tokens = re.findall(r'\b\w+\b', text)
    
    for token in tokens:
        if token not in stop_words:
            filtered_tokens.append(token)
        else:
            filtered_tokens.append(" - ")
            
    return filtered_tokens

In [80]:
tokenss = remove_stopwordss(tokens)
tokens = remove_stopwords(tokens)

data['Final Result'] = tokenss.copy()


In [81]:
finaltext = ' '.join(tokens)
data = pd.DataFrame(data)

In [82]:
print("--------------------------------------")
print("Text Before Stemming")
print("--------------------------------------")
print(text)
print("--------------------------------------")
print("Text After Stemming")
print("--------------------------------------")
print(finaltext)
print("--------------------------------------")

--------------------------------------
Text Before Stemming
--------------------------------------
الإيمانُ ..وأثره على الصحة النفسية 
كان علم النفس دائماً فخوراً كعلم بالجامعات بتقاليده الدنيوية )غير
الدينية( الملتزمة بالتنوير. وكان -على الدوام- من ضمن هذه
التقاليد وجود شك واضح بكل أشكال التدين  كما يصف بيرنارد
وغروم
--------------------------------------
Text After Stemming
--------------------------------------
إيمان اثره صحه علم خور علم جامعه تقاليد دنيوي ديني ملتزم تنوير دوام تقاليد جود شك اضح شكال تدين صف يرنارد غروم
--------------------------------------


In [83]:
data

,Tokens,Normalized Tokens,Suffex Removed,Prefex Removed,Last Suffex Removed,Final Result
0,الإيمان,الإيمان,الإيمان,إيمان,إيمان,إيمان
1,وأثره,واثره,واثره,اثره,اثره,اثره
2,الصحة,الصحه,الصحه,صحه,صحه,صحه
3,النفسية,النفسيه,النفسي,نفسي,نفسي,-
4,علم,علم,علم,علم,علم,علم
5,النفس,النفس,النفس,نفس,نفس,-
6,دائما,دائما,دائما,دائما,دائم,-
7,فخورا,فخورا,فخورا,خورا,خور,خور
8,كعلم,كعلم,كعلم,علم,علم,علم
9,بالجامعات,بالجامعات,بالجامعه,جامعه,جامعه,جامعه
